# Introduction Instrument Class

In [1]:
# SHow how to calculate numerical Greeks when the engine doesn't provide them

import QuantLib as ql

today = ql.Date(8, ql.October, 2014)
ql.Settings.instance().evaluationDate = today

In [2]:
# a knock-in barrier option example

option = ql.BarrierOption(ql.Barrier.UpIn,
                          120.0, #barrier
                          0.0,   #rebate
                          ql.PlainVanillaPayoff(ql.Option.Call, 100.0),
                          ql.EuropeanExercise(ql.Date(8, ql.January, 2015)))

#for the purpose of this example, the market data are the underlying value, the risk-free rate and the volatility.
#we wrap them in quotes, so that the instrument will be notified of any changes.

u     = ql.SimpleQuote(100.0)
r     = ql.SimpleQuote(0.01)
sigma = ql.SimpleQuote(0.20)

In [5]:
# from the quotes we build the flat curves and the process that the engine requires.
# we build the term structures so that they move with the evaluation date

riskFreeCurve    = ql.FlatForward(0, 
                                  ql.TARGET(),
                                  ql.QuoteHandle(r),
                                  ql.Actual360())

volatility       = ql.BlackConstantVol(0,
                                       ql.TARGET(),
                                       ql.QuoteHandle(sigma),
                                       ql.Actual360())

process          = ql.BlackScholesProcess(ql.QuoteHandle(u),
                                          ql.YieldTermStructureHandle(riskFreeCurve),
                                          ql.BlackVolTermStructureHandle(volatility))

# Build the engine (the library provides one based on a analyti formula) and set it to the option.

option.setPricingEngine(ql.AnalyticBarrierEngine(process))

# ask the option for its value
print(option.NPV())

1.3657980739109867


In [6]:
print(option.delta())

RuntimeError: delta not provided

# Numberical calculation of Greeks

In [7]:
u0 = u.value()    ; h = 0.01   #underlying and step
P0 = option.NPV() ; print(P0)  #option price

1.3657980739109867


In [8]:
# increase the underlying value and get the new option value

u.setValue(u0 + h)
P_plus = option.NPV() ; print(P_plus)

1.3688112201958078


In [9]:
u.setValue(u0 - h)
P_minus = option.NPV() ; print(P_minus)

1.3627900998610203


In [10]:
# Set the underlying value back to its current value

u.setValue(u0)

# Applying the formulas above give us the desired Greeks

Delta = (P_plus - P_minus)/(2*h)
Gamma = (P_plus - 2*P0 + P_minus)/(h*h)

print(Delta)
print(Gamma)

0.3010560167393761
0.05172234854633473


In [11]:
# calculate Rho and Vega

r0 = r.value(); h = 0.0001
r.setValue(r0+h) ; P_plus = option.NPV()
r.setValue(r0)
Rho = (P_plus - P0)/h ; print(Rho)

6.531038494281827


In [12]:
sigma0 = sigma.value() ; h = 0.0001
sigma.setValue(sigma0+h) ; P_plus = option.NPV()
sigma.setValue(sigma0)
Vega = (P_plus - P0)/h ; print(Vega)

26.52519924198904


In [13]:
# The approach for the Theta is bit different

ql.Settings.instance().evaluationDate = today + 1
P1 = option.NPV()
h  = 1.0/365
Theta = (P1 - P0)/h ; print(Theta)

-10.770888399441302
